## Scenario 2: A cross-functional team with one data scientist working on an ML model

This scenario demonstrates how a data scientist can use MLflow to track machine learning experiments in a team setting, using a centralized tracking server. This setup is common in organizations where multiple people need to access experiment results, models, and artifacts.

### MLflow setup overview:
- **Tracking server:** Yes (runs as a local server, accessible to the team)
- **Backend store:** SQLite database (stores experiment metadata in `backend.db`)
- **Artifacts store:** Local filesystem (stores model files and other artifacts)

With this setup, all experiment runs, parameters, metrics, and artifacts are saved in a central location. Team members can explore and compare experiments using the MLflow UI, even from different machines (if the server is accessible).

### How to use the MLflow tracking server and UI
- **First, you must launch the MLflow tracking server** by running the following command in your terminal:
  ```bash
  mlflow server --backend-store-uri sqlite:///backend.db
  ```
- The UI will be available at the address printed in your terminal (by default, [http://localhost:5000](http://localhost:5000)).
- If you run the server on a remote machine or a different port, use the appropriate address (e.g., `http://<your-server>:<port>`).
- Use the UI to browse experiments, compare runs, and inspect logged models and artifacts.
- You can also interact with the model registry for collaborative model management.

> **Tip:** This setup is ideal for small teams and collaborative projects. For larger teams or production, you may use a remote database and cloud storage for the backend and artifacts.

In [1]:
import mlflow


mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [2]:
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: 'http://127.0.0.1:5000'


In [3]:
mlflow.search_experiments()

[<Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1761735682063, experiment_id='0', last_update_time=1761735682063, lifecycle_stage='active', name='Default', tags={}>]

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score, confusion_matrix
import numpy as np
import mlflow
import sklearn
import datetime
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

mlflow.set_experiment("my-experiment-1")

X, y = load_iris(return_X_y=True)
class_names = load_iris().target_names

# Try several values of C to demonstrate experiment tracking
for C in [0.01, 0.1, 1, 10]:
    params = {"C": C, "random_state": 42, "max_iter": 1000, "solver": "lbfgs"}
    with mlflow.start_run() as run:
        mlflow.log_params(params)
        mlflow.log_param("sklearn_version", sklearn.__version__)
        lr = LogisticRegression(**params).fit(X, y)
        y_pred = lr.predict(X)
        acc = accuracy_score(y, y_pred)
        mlflow.log_metric("accuracy", acc)
        # Log model coefficients as params (flattened for logging)
        for i, coef in enumerate(lr.coef_.flatten()):
            mlflow.log_param(f"coef_{i}", coef)
        # Labeled confusion matrix as DataFrame
        cm = confusion_matrix(y, y_pred)
        cm_df = pd.DataFrame(cm, index=class_names, columns=class_names)
        cm_df.to_csv("confusion_matrix_labeled.csv")
        mlflow.log_artifact("confusion_matrix_labeled.csv")
        # Confusion matrix heatmap as image
        plt.figure(figsize=(5,4))
        sns.heatmap(cm_df, annot=True, fmt="d", cmap="Blues")
        plt.title(f"Confusion Matrix (C={C})")
        plt.ylabel("True label")
        plt.xlabel("Predicted label")
        plt.tight_layout()
        plt.savefig("confusion_matrix_heatmap.png")
        plt.close()
        mlflow.log_artifact("confusion_matrix_heatmap.png")
        # Provide input_example and use 'name' instead of deprecated 'artifact_path'
        input_example = np.expand_dims(X[0], axis=0)
        mlflow.sklearn.log_model(lr, name="models", input_example=input_example)
        # Log model type, number of classes, and timestamp as tags
        mlflow.set_tag("model_type", type(lr).__name__)
        mlflow.set_tag("n_classes", len(np.unique(y)))
        mlflow.set_tag("run_time", datetime.datetime.now().isoformat())
        mlflow.set_tag("description", "Logistic regression on Iris dataset with varying C")
        print(f"Logged run for C={C}, accuracy={acc:.3f}")

2025/10/29 12:10:51 INFO mlflow.tracking.fluent: Experiment with name 'my-experiment-1' does not exist. Creating a new experiment.
c:\demo\ie-mlops-nyc-taxis\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Logged run for C=0.01, accuracy=0.873
🏃 View run adaptable-fish-252 at: http://127.0.0.1:5000/#/experiments/1/runs/a492e384454a4859862ccead60c48ff0
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1


Logged run for C=0.1, accuracy=0.960
🏃 View run delicate-koi-472 at: http://127.0.0.1:5000/#/experiments/1/runs/c6ce29050c40498aa75ff4cbf841778e
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1


Logged run for C=1, accuracy=0.973
🏃 View run treasured-zebra-284 at: http://127.0.0.1:5000/#/experiments/1/runs/aab555ae428d40e9a86b7e33a8509005
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1


Logged run for C=10, accuracy=0.980
🏃 View run selective-vole-73 at: http://127.0.0.1:5000/#/experiments/1/runs/317d008ca52c485c8b4eec4252302a8f
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1


In [5]:
mlflow.search_experiments()

[<Experiment: artifact_location='mlflow-artifacts:/1', creation_time=1761736251672, experiment_id='1', last_update_time=1761736251672, lifecycle_stage='active', name='my-experiment-1', tags={'mlflow.experimentKind': 'custom_model_development'}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1761735682063, experiment_id='0', last_update_time=1761735682063, lifecycle_stage='active', name='Default', tags={}>]

### Interacting with the model registry

In [6]:
from mlflow.tracking import MlflowClient


client = MlflowClient("http://127.0.0.1:5000")

In [7]:
client.search_registered_models()

[]

In [8]:
run_id = client.search_runs(experiment_ids='1')[0].info.run_id
mlflow.register_model(
    model_uri=f"runs:/{run_id}/models",
    name='iris-classifier'
)

Successfully registered model 'iris-classifier'.
2025/10/29 12:14:20 WARNING mlflow.tracking._model_registry.fluent: Run with id 317d008ca52c485c8b4eec4252302a8f has no artifacts at artifact path 'models', registering model based on models:/m-4b8166273b7341c683a89ec67694f2ae instead
2025/10/29 12:14:22 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: iris-classifier, version 1
Created version '1' of model 'iris-classifier'.


<ModelVersion: aliases=[], creation_timestamp=1761736462668, current_stage='None', deployment_job_state=<ModelVersionDeploymentJobState: current_task_name='', job_id='', job_state='DEPLOYMENT_JOB_CONNECTION_STATE_UNSPECIFIED', run_id='', run_state='DEPLOYMENT_JOB_RUN_STATE_UNSPECIFIED'>, description='', last_updated_timestamp=1761736462668, metrics=None, model_id=None, name='iris-classifier', params=None, run_id='317d008ca52c485c8b4eec4252302a8f', run_link='', source='models:/m-4b8166273b7341c683a89ec67694f2ae', status='READY', status_message=None, tags={}, user_id='', version='1'>